In [ ]:
import pandas as pd
import transformers
import torch
import random
import pandas as pd
import random
import string
import numpy as np
import unidecode
from unidecode import unidecode
from wordcloud import STOPWORDS
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, f1_score
from torch.utils.data import TensorDataset
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoTokenizer, BertModel, BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, PreTrainedTokenizerFast
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
from sklearn import metrics
import tensorflow as tf
from tqdm.notebook import tqdm
import numpy as np
import torch
transformers.logging.set_verbosity_debug()
from loguru import logger
logger.add("logging/finnlp_bert_undersampling_experiment_1.log", rotation="500 MB")

In [15]:
BATCH_SIZE = 20

In [16]:
logger.debug("<<<<<<<<<< INICIOU >>>>>>>>>>>")
brc = pd.read_csv("Banking_Regulation_Corpora_BRC_anonymous.csv" ,encoding="utf-8", low_memory=False, sep=";")
data_relevant = brc[brc['class'] == 1]
data_irrelevant = brc[brc['class'] == 0]

2024-03-01 20:36:15.942 | DEBUG    | __main__:<module>:1 - <<<<<<<<<< INICIOU >>>>>>>>>>>


In [17]:
data_relevant.count()

class                   7846
department              7846
entry_date              7846
general_id              7846
normative_identifier    7846
publication_date        7846
regulatory_authority    7846
subject                 7846
subject_length          7846
subject_unique_words    7846
subject_words           7846
text                    7846
text_length             7846
text_unique_words       7846
text_words              7846
type                    7846
unique_document_id      7846
url                     7846
dtype: int64

In [18]:
data_relevant = data_relevant[data_relevant.text_words >= 50]
data_relevant.count()

class                   7714
department              7714
entry_date              7714
general_id              7714
normative_identifier    7714
publication_date        7714
regulatory_authority    7714
subject                 7714
subject_length          7714
subject_unique_words    7714
subject_words           7714
text                    7714
text_length             7714
text_unique_words       7714
text_words              7714
type                    7714
unique_document_id      7714
url                     7714
dtype: int64

In [19]:
data_relevant['class'] = data_relevant['class'].astype(str)

In [20]:
data_relevant = data_relevant.rename(columns={"class":"label"})

In [21]:
data_relevant.count()

label                   7714
department              7714
entry_date              7714
general_id              7714
normative_identifier    7714
publication_date        7714
regulatory_authority    7714
subject                 7714
subject_length          7714
subject_unique_words    7714
subject_words           7714
text                    7714
text_length             7714
text_unique_words       7714
text_words              7714
type                    7714
unique_document_id      7714
url                     7714
dtype: int64

In [22]:
data_irrelevant.count()

class                   54018
department              54018
entry_date              54018
general_id              54018
normative_identifier    54018
publication_date        54018
regulatory_authority    54018
subject                 54018
subject_length          54018
subject_unique_words    54018
subject_words           54018
text                    54018
text_length             54018
text_unique_words       54018
text_words              54018
type                    54018
unique_document_id      54018
url                     54018
dtype: int64

In [23]:
data_irrelevant = data_irrelevant[data_irrelevant.text_words >= 50]
data_irrelevant.count()

class                   53122
department              53122
entry_date              53122
general_id              53122
normative_identifier    53122
publication_date        53122
regulatory_authority    53122
subject                 53122
subject_length          53122
subject_unique_words    53122
subject_words           53122
text                    53122
text_length             53122
text_unique_words       53122
text_words              53122
type                    53122
unique_document_id      53122
url                     53122
dtype: int64

In [24]:
data_irrelevant['class'] = data_irrelevant['class'].astype(str)

In [25]:
data_irrelevant = data_irrelevant.rename(columns={"class":"label"})

In [26]:
data_irrelevant.count()

label                   53122
department              53122
entry_date              53122
general_id              53122
normative_identifier    53122
publication_date        53122
regulatory_authority    53122
subject                 53122
subject_length          53122
subject_unique_words    53122
subject_words           53122
text                    53122
text_length             53122
text_unique_words       53122
text_words              53122
type                    53122
unique_document_id      53122
url                     53122
dtype: int64

In [27]:
class Model_BERT():
    
    def model_path_def(self, name_model):
        if name_model == "BERTIMBAU_BASE":
            model_path = MODEL_PATH
        elif name_model == "XXX":
            model_path = MODEL_PATH_NEW
        else:
            print("Nome de modelo inválido")
            model_path = "None"
            
        return model_path

    def tokenizer_model(self, model_path, df, token):
        tokenizer = BertTokenizer.from_pretrained(BERTIMBAU_TOKENIZER, do_lower_case=True)

        # Tokenizing the training data
        encoded_data_train = tokenizer.batch_encode_plus(
            df[df.data_type=='train'].new_content.tolist(), 
            add_special_tokens=True, 
            return_attention_mask=True, 
            padding=True, 
            max_length=token, 
            return_tensors='pt',
            truncation=True
        )

        # Tokenizing test data
        encoded_data_val = tokenizer.batch_encode_plus(
            df[df.data_type=='val'].new_content.tolist(), 
            add_special_tokens=True, 
            return_attention_mask=True, 
            padding=True,
            max_length=token, 
            return_tensors='pt',
            truncation=True
        )


        input_ids_train = encoded_data_train['input_ids']
        attention_masks_train = encoded_data_train['attention_mask']
        labels_train = torch.tensor(df[df.data_type=='train'].label_num.values)

        input_ids_val = encoded_data_val['input_ids']
        attention_masks_val = encoded_data_val['attention_mask']
        labels_val = torch.tensor(df[df.data_type=='val'].label_num.values)

        dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
        dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
        
        return(dataset_train, dataset_val)

    def setup_model(self, model_path, dataset_train, dataset_val, epochs):
        model = BertForSequenceClassification.from_pretrained(model_path,
                                                              num_labels=len(LABEL_DICT),
                                                              output_attentions=False,
                                                              output_hidden_states=False)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)
        batch_size = BATCH_SIZE
        dataloader_train = DataLoader(dataset_train, 
                                      sampler=RandomSampler(dataset_train), 
                                      batch_size=batch_size)
        dataloader_validation = DataLoader(dataset_val, 
                                           sampler=SequentialSampler(dataset_val), 
                                           batch_size=batch_size)
        optimizer = AdamW(model.parameters(),
                          lr=1e-5, 
                          eps=1e-8)

        scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                    num_warmup_steps=100,
                                                    num_training_steps=len(dataloader_train)*epochs)
        return(model, dataloader_train, dataloader_validation, epochs, scheduler, optimizer, device)


    def f1_score_func(self, preds, labels):
        preds_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()
        
        scr = f1_score(labels_flat, preds_flat, average='binary')
        logger.info(f"f1_score_binary: {scr}")
        return scr

    def accuracy_per_class(self, preds, labels):
        label_dict_inverse = {v: k for k, v in LABEL_DICT.items()}
        
        preds_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()

        for label in np.unique(labels_flat):
            y_preds = preds_flat[labels_flat==label]
            y_true = labels_flat[labels_flat==label]
            print(f'Class: {label_dict_inverse[label]}')
            print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')
            print(f'% Accuracy: {len(y_preds[y_preds==label])/len(y_true)}\n')
            logger.info(f'Class: {label_dict_inverse[label]}')
            logger.info(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')
            logger.info(f'% Accuracy: {len(y_preds[y_preds==label])/len(y_true)}\n')
        
    def evaluate(self, model, dataloader_val, device):

        model.eval()

        loss_val_total = 0
        predictions, true_vals = [], []

        for batch in dataloader_val:

            batch = tuple(b.to(device) for b in batch)

            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':         batch[2],
                     }

            with torch.no_grad():        
                outputs = model(**inputs)

            loss = outputs[0]
            logits = outputs[1]
            loss_val_total += loss.item()

            logits = logits.detach().cpu().numpy()
            label_ids = inputs['labels'].cpu().numpy()
            predictions.append(logits)
            true_vals.append(label_ids)

        loss_val_avg = loss_val_total/len(dataloader_val) 

        predictions = np.concatenate(predictions, axis=0)
        true_vals = np.concatenate(true_vals, axis=0)

        logger.info(f"loss_val_avg, predictions, true_vals: {loss_val_avg}, {predictions}, {true_vals}")

        return loss_val_avg, predictions, true_vals

    def results(self, model_path, model_name, epochs, dataloader_validation, device, token):
        model = BertForSequenceClassification.from_pretrained(model_path,
                                                              num_labels=len(LABEL_DICT),
                                                              output_attentions=False,
                                                              output_hidden_states=False)

        model.to(device)

        model.load_state_dict(torch.load(f'{MODEL_SAVE_PATH}/token{token}_model_data_{model_name}_epoch_{epochs}.model', map_location=torch.device('cpu')))

        _, predictions, true_vals = Model_BERT().evaluate(model, dataloader_validation, device)
        Model_BERT().accuracy_per_class(predictions, true_vals)
        from sklearn import metrics
        preds_flat = np.argmax(predictions, axis=1).flatten()
        metrics_report = metrics.classification_report(true_vals,preds_flat, output_dict=True)
        logger.info(metrics_report)
        
        return metrics_report
    

    def training_model(self, model, model_name, dataloader_train, dataloader_validation, epochs, scheduler, optimizer, device, token):
        seed_val = 1
        random.seed(seed_val)
        np.random.seed(seed_val)
        torch.manual_seed(seed_val)
        torch.cuda.manual_seed_all(seed_val)
        epoch_array = []
        f1_array = []
        train_loss_array = []
        val_loss_array = []

        for epoch in tqdm(range(1, epochs+1)):

            model.train()

            loss_train_total = 0

            progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
            for batch in progress_bar:

                model.zero_grad()

                batch = tuple(b.to(device) for b in batch)

                inputs = {'input_ids':      batch[0],
                          'attention_mask': batch[1],
                          'labels':         batch[2],
                         }       

                outputs = model(**inputs)

                loss = outputs[0]
                loss_train_total += loss.item()
                loss.backward()

                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

                optimizer.step()
                scheduler.step()

                progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})


            torch.save(model.state_dict(),f'{MODEL_SAVE_PATH}/token{token}_model_data_{model_name}_epoch_{epoch}.model')
            model.save_pretrained(f"{MODEL_SAVE_PATH}/token{token}_model_data_{model_name}_epoch_{epoch}")
            tqdm.write(f'\nEpoch {epoch}')
            print(f'\nEpoch {epoch}')
            logger.info(f'\nEpoch {epoch}')

            loss_train_avg = loss_train_total/len(dataloader_train)            
            tqdm.write(f'Training loss: {loss_train_avg}')
            logger.info(f'Training loss: {loss_train_avg}')

            val_loss, predictions, true_vals = Model_BERT().evaluate(model, dataloader_validation, device)
            val_f1 = Model_BERT().f1_score_func(predictions, true_vals)
            tqdm.write(f'Validation loss: {val_loss}')
            print(f'Validation loss: {val_loss}')
            logger.info(f'Validation loss: {val_loss}')
            
            tqdm.write(f'F1 Score (Weighted/Binary): {val_f1}')
            print(f'F1 Score (Weighted/Binary): {val_f1}')
            logger.info(f'F1 Score (Weighted/Binary): {val_f1}')
            
            epoch_array.append(epoch)
            f1_array.append(val_f1)
            val_loss_array.append(val_loss)
            train_loss_array.append(loss_train_avg)

        return(epoch_array, f1_array, val_loss_array, train_loss_array)
    
    def read_df(self, df):
        df['label_num'] = df.label.replace(LABEL_DICT)

        X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label_num.values, 
                                                  test_size=0.4, 
                                                  random_state=42, 
                                                  stratify=df.label.values)
        df['data_type'] = ['not_set']*df.shape[0]
        df.loc[X_train, 'data_type'] = 'train'
        df.loc[X_val, 'data_type'] = 'val'
        
        return(df)

    def call_setup(self, model_name, df, token, epochs):
        df = Model_BERT().read_df(df)
        model_path = Model_BERT().model_path_def(model_name)
        print(model_path)
        dataset_train, dataset_val = Model_BERT().tokenizer_model(model_path, df, token)
        model, dataloader_train, dataloader_val, epochs, scheduler, optimizer, device = Model_BERT().setup_model(model_path, dataset_train, dataset_val, epochs)
        epoch_array, f1_array, val_loss_array, train_loss_array = Model_BERT().training_model(model, model_name, dataloader_train, dataloader_val, epochs, scheduler, optimizer, device, token)
        df_performa = pd.DataFrame()
        df_performa["epoch"] = epoch_array
        df_performa["f1"] = f1_array
        df_performa["val_loss"] = val_loss_array
        df_performa["train_loss"] = train_loss_array
        metrics_report = Model_BERT().results(model_path,model_name,epochs,dataloader_val, device, token)
        logger.info(f'Metric Report Call Setup: {metrics_report}')
        
        return metrics_report, df_performa

In [28]:
departments = ['DPT_25', 'DPT_3', 'DPT_2', 'DPT_9', 'DPT_5', 'DPT_16']

In [29]:
def cleaning(df,column):
    # Remove urls
    df.loc[:,column] = df[column].replace(r'https?:\/\/.*?[\s+]', ' ', regex=True)
    # Remove urls without https?
    df.loc[:,column] = df[column].replace(r'www.*?[\s+]', ' ', regex=True)
    # Remove email address
    df.loc[:,column] = df[column].replace(r'(?P<email_address>[\w\.-]+@[\w\.-]+\.[\w]+)', ' ', regex=True)
    
    df.loc[:,column] = df[column].str.replace('º',' ')
    df.loc[:,column] = df[column].str.replace('ª',' ')

    df.loc[:,column] = df[column].str.replace('1',' ')
    df.loc[:,column] = df[column].str.replace('2',' ')
    df.loc[:,column] = df[column].str.replace('3',' ')
    df.loc[:,column] = df[column].str.replace('4',' ')
    df.loc[:,column] = df[column].str.replace('5',' ')
    df.loc[:,column] = df[column].str.replace('6',' ')
    df.loc[:,column] = df[column].str.replace('7',' ')
    df.loc[:,column] = df[column].str.replace('8',' ')
    df.loc[:,column] = df[column].str.replace('9',' ')
    df.loc[:,column] = df[column].str.replace('0',' ')

    df.loc[:,column] = df[column].str.replace('/',' ')

    df.loc[:,column] = df[column].str.replace('\r',' ')
    df.loc[:,column] = df[column].str.replace('\n',' ')
    df.loc[:,column] = df[column].str.replace('\t',' ')
    df.loc[:,column] = df[column].str.replace('\\',' ')

    df.loc[:,column] = df[column].str.replace('-',' ')

    df.loc[:,column] = df[column].str.replace('–',' ')
    df.loc[:,column] = df[column].str.replace('“',' ')
    df.loc[:,column] = df[column].str.replace('”',' ')
    df.loc[:,column] = df[column].str.replace('’',' ')
    df.loc[:,column] = df[column].str.replace('_',' ')
    df.loc[:,column] = df[column].str.replace('.',' ')
    df.loc[:,column] = df[column].str.replace(',',' ')
    df.loc[:,column] = df[column].str.replace('|',' ')
    df.loc[:,column] = df[column].str.replace('=',' ')
    df.loc[:,column] = df[column].str.replace('@',' ')
    df.loc[:,column] = df[column].str.replace('$',' ')

    df.loc[:,column] = df[column].str.replace('°',' ')
    df.loc[:,column] = df[column].str.replace('§',' ')
    df.loc[:,column] = df[column].str.replace('•',' ')
    df.loc[:,column] = df[column].str.replace('▪',' ')

    df.loc[:,column] = df[column].str.replace('%',' ')
    df.loc[:,column] = df[column].str.replace('&',' ')
    df.loc[:,column] = df[column].str.replace('*',' ')
    df.loc[:,column] = df[column].str.replace('+',' ')
    df.loc[:,column] = df[column].str.replace(':',' ')
    df.loc[:,column] = df[column].str.replace(';',' ')

    df.loc[:,column] = df[column].str.replace('!',' ')
    df.loc[:,column] = df[column].str.replace('?',' ')
    df.loc[:,column] = df[column].str.replace('#',' ')
    df.loc[:,column] = df[column].str.replace('\'',' ')
    df.loc[:,column] = df[column].str.replace('"',' ')

    df.loc[:,column] = df[column].str.replace('<',' ')
    df.loc[:,column] = df[column].str.replace('>',' ')

    df.loc[:,column] = df[column].str.replace('(',' ')
    df.loc[:,column] = df[column].str.replace(')',' ')

    df.loc[:,column] = df[column].str.replace('{',' ')
    df.loc[:,column] = df[column].str.replace('}',' ')

    df.loc[:,column] = df[column].str.replace('[',' ')
    df.loc[:,column] = df[column].str.replace(']',' ')

    # Remove the same character repeated more than twice
    df.loc[:,column] = df[column].replace(r'([a-z])\1{2,}', ' ', regex=True)
    
    # To replace more than one white space by only one white space
    df.loc[:,column] = df[column].replace(r'\s+', ' ', regex=True)

    return df

In [ ]:
for department in departments:

    total_relevant_documents_of_the_board = data_relevant[data_relevant.department == department].shape[0]
    logger.info(f"total_relevant_documents_of_the_board: {total_relevant_documents_of_the_board} - {department}")
    logger.info(f"Total irrelevant documents of the department: {data_irrelevant[data_irrelevant.department == department].shape[0]} - {department}")
    
    data_relevant_board = data_relevant[data_relevant.department == department]
    logger.info(f"data_relevant_board: {data_relevant_board.shape} - {department}")
    
    # The total documents of the relevant class (total_relevant_documents_of_the_board) is use to get the first documents in the irrelevant class.
    data_irrelevant_board = data_irrelevant[data_irrelevant.department == department][:total_relevant_documents_of_the_board]
    logger.info(f"data_irrelevant_board: {data_irrelevant_board.shape} - {department}")

    df_ace_modelagem = pd.concat([data_irrelevant_board, data_relevant_board])

    logger.debug(f"Department: {department}")
    df_modelagem = df_ace_modelagem[df_ace_modelagem.department == department]

    df_modelagem = df_modelagem.dropna()
    df_modelagem = df_modelagem.reset_index(drop=True)
    df_modelagem = df_modelagem.rename(columns={"text":"content"})

    df_modelagem = cleaning(df_modelagem,"content")

    new_content = []
    for texto in df_modelagem.content:
        new_content.append(texto.replace("[BUSINESS NEED]", ""))
    df_modelagem["new_content"] = new_content
    
    LABEL_DICT = {'0': 0, '1': 1}

    MODEL_SAVE_PATH = "notebooks/testes/scoring"
    BERTIMBAU_BASE = "datasets/BERTIMBAU_BASE"
    BERTIMBAU_TOKENIZER = "tokenizer/BERTimbau_Tokenizer"

    MODEL_PATH = BERTIMBAU_BASE
    MODEL_PATH_NEW = BERTIMBAU_BASE

    tokenizer = BertTokenizer.from_pretrained(BERTIMBAU_TOKENIZER, do_lower_case=True)

    logger.debug(f"Department count samples: {df_modelagem.count()}")
    logger.debug(f"Department count samples class 1: {df_modelagem[df_modelagem.label == '1'].count()}")
    logger.debug(f"Department count samples class 0: {df_modelagem[df_modelagem.label == '0'].count()}")
    

    Model_BERT().call_setup("BERTIMBAU_BASE", df_modelagem, 512, 5)

<h2>The output result is in the next cell. It was copied from the log file.</h2>

In [ ]:
# 2023-10-09 14:40:52.799 | DEBUG    | __main__:<module>:1 - <<<<<<<<<< INICIOU >>>>>>>>>>>
# 2023-10-09 14:41:02.818 | INFO     | __main__:<module>:4 - total_relevant_documents_of_the_board: 819 - DPT_25
# 2023-10-09 14:41:02.824 | INFO     | __main__:<module>:5 - Total irrelevant documents of the department: 4051 - DPT_25
# 2023-10-09 14:41:02.825 | INFO     | __main__:<module>:8 - data_relevant_board: (819, 18) - DPT_25
# 2023-10-09 14:41:02.830 | INFO     | __main__:<module>:12 - data_irrelevant_board: (819, 18) - DPT_25
# 2023-10-09 14:41:02.834 | DEBUG    | __main__:<module>:18 - Department: DPT_25
# 2023-10-09 14:41:06.088 | DEBUG    | __main__:<module>:56 - Department count samples: label                   1638
# department              1638
# entry_date              1638
# general_id              1638
# normative_identifier    1638
# publication_date        1638
# regulatory_authority    1638
# subject                 1638
# subject_length          1638
# subject_unique_words    1638
# subject_words           1638
# content                 1638
# text_length             1638
# text_unique_words       1638
# text_words              1638
# type                    1638
# unique_document_id      1638
# url                     1638
# new_content             1638
# dtype: int64
# 2023-10-09 14:41:06.092 | DEBUG    | __main__:<module>:57 - Department count samples class 1: label                   819
# department              819
# entry_date              819
# general_id              819
# normative_identifier    819
# publication_date        819
# regulatory_authority    819
# subject                 819
# subject_length          819
# subject_unique_words    819
# subject_words           819
# content                 819
# text_length             819
# text_unique_words       819
# text_words              819
# type                    819
# unique_document_id      819
# url                     819
# new_content             819
# dtype: int64
# 2023-10-09 14:41:06.095 | DEBUG    | __main__:<module>:58 - Department count samples class 0: label                   819
# department              819
# entry_date              819
# general_id              819
# normative_identifier    819
# publication_date        819
# regulatory_authority    819
# subject                 819
# subject_length          819
# subject_unique_words    819
# subject_words           819
# content                 819
# text_length             819
# text_unique_words       819
# text_words              819
# type                    819
# unique_document_id      819
# url                     819
# new_content             819
# dtype: int64
# 2023-10-09 14:42:41.071 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-09 14:42:41.072 | INFO     | __main__:training_model:214 - Training loss: 0.6758278459310532
# 2023-10-09 14:42:47.033 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6233953806486997, [[ 0.00348926 -0.07517872]
 
# 2023-10-09 14:42:47.037 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.7463235294117647
# 2023-10-09 14:42:47.038 | INFO     | __main__:training_model:220 - Validation loss: 0.6233953806486997
# 2023-10-09 14:42:47.039 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.7463235294117647
# 2023-10-09 14:43:17.004 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-09 14:43:17.005 | INFO     | __main__:training_model:214 - Training loss: 0.553212727420032
# 2023-10-09 14:43:23.029 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.4812545518983494, [[-0.5217643   0.438042  ]

# 2023-10-09 14:43:23.032 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8144329896907216
# 2023-10-09 14:43:23.034 | INFO     | __main__:training_model:220 - Validation loss: 0.4812545518983494
# 2023-10-09 14:43:23.035 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8144329896907216
# 2023-10-09 14:43:53.028 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-09 14:43:53.030 | INFO     | __main__:training_model:214 - Training loss: 0.4131041709333658
# 2023-10-09 14:43:59.070 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.36204276098446414, [[-1.215345    1.0647657 ]

# 2023-10-09 14:43:59.074 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8641571194762684
# 2023-10-09 14:43:59.076 | INFO     | __main__:training_model:220 - Validation loss: 0.36204276098446414
# 2023-10-09 14:43:59.077 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8641571194762684
# 2023-10-09 14:44:29.141 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-09 14:44:29.143 | INFO     | __main__:training_model:214 - Training loss: 0.3442826443351805
# 2023-10-09 14:44:35.186 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.334580886093053, [[-1.4477183   1.4226424 ]

# 2023-10-09 14:44:35.190 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.869281045751634
# 2023-10-09 14:44:35.192 | INFO     | __main__:training_model:220 - Validation loss: 0.334580886093053
# 2023-10-09 14:44:35.193 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.869281045751634
# 2023-10-09 14:45:05.103 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-09 14:45:05.105 | INFO     | __main__:training_model:214 - Training loss: 0.3133479580283165
# 2023-10-09 14:45:11.149 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3216054520823739, [[-1.5153731   1.5040814 ]

# 2023-10-09 14:45:11.153 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8752025931928687
# 2023-10-09 14:45:11.155 | INFO     | __main__:training_model:220 - Validation loss: 0.3216054520823739
# 2023-10-09 14:45:11.156 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8752025931928687
# 2023-10-09 14:45:18.793 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3216054520823739, [[-1.5153731   1.5040814 ]

# 2023-10-09 14:45:18.795 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-09 14:45:18.795 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 309/328
# 2023-10-09 14:45:18.796 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.9420731707317073

# 2023-10-09 14:45:18.797 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-09 14:45:18.798 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 270/328
# 2023-10-09 14:45:18.799 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.823170731707317

# 2023-10-09 14:45:18.807 | INFO     | __main__:results:157 - {'0': {'precision': 0.8419618528610354, 'recall': 0.9420731707317073, 'f1-score': 0.889208633093525, 'support': 328}, '1': {'precision': 0.9342560553633218, 'recall': 0.823170731707317, 'f1-score': 0.8752025931928687, 'support': 328}, 'accuracy': 0.8826219512195121, 'macro avg': {'precision': 0.8881089541121786, 'recall': 0.8826219512195121, 'f1-score': 0.8822056131431969, 'support': 656}, 'weighted avg': {'precision': 0.8881089541121786, 'recall': 0.8826219512195121, 'f1-score': 0.8822056131431968, 'support': 656}}
# 2023-10-09 14:45:18.809 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.8419618528610354, 'recall': 0.9420731707317073, 'f1-score': 0.889208633093525, 'support': 328}, '1': {'precision': 0.9342560553633218, 'recall': 0.823170731707317, 'f1-score': 0.8752025931928687, 'support': 328}, 'accuracy': 0.8826219512195121, 'macro avg': {'precision': 0.8881089541121786, 'recall': 0.8826219512195121, 'f1-score': 0.8822056131431969, 'support': 656}, 'weighted avg': {'precision': 0.8881089541121786, 'recall': 0.8826219512195121, 'f1-score': 0.8822056131431968, 'support': 656}}
# 2023-10-09 14:45:18.814 | INFO     | __main__:<module>:4 - total_relevant_documents_of_the_board: 688 - DPT_3
# 2023-10-09 14:45:18.825 | INFO     | __main__:<module>:5 - Total irrelevant documents of the department: 1790 - DPT_3
# 2023-10-09 14:45:18.827 | INFO     | __main__:<module>:8 - data_relevant_board: (688, 18) - DPT_3
# 2023-10-09 14:45:18.832 | INFO     | __main__:<module>:12 - data_irrelevant_board: (688, 18) - DPT_3
# 2023-10-09 14:45:18.836 | DEBUG    | __main__:<module>:18 - Department: DPT_3
# 2023-10-09 14:45:32.081 | DEBUG    | __main__:<module>:56 - Department count samples: label                   1376
# department              1376
# entry_date              1376
# general_id              1376
# normative_identifier    1376
# publication_date        1376
# regulatory_authority    1376
# subject                 1376
# subject_length          1376
# subject_unique_words    1376
# subject_words           1376
# content                 1376
# text_length             1376
# text_unique_words       1376
# text_words              1376
# type                    1376
# unique_document_id      1376
# url                     1376
# new_content             1376
# dtype: int64
# 2023-10-09 14:45:32.085 | DEBUG    | __main__:<module>:57 - Department count samples class 1: label                   688
# department              688
# entry_date              688
# general_id              688
# normative_identifier    688
# publication_date        688
# regulatory_authority    688
# subject                 688
# subject_length          688
# subject_unique_words    688
# subject_words           688
# content                 688
# text_length             688
# text_unique_words       688
# text_words              688
# type                    688
# unique_document_id      688
# url                     688
# new_content             688
# dtype: int64
# 2023-10-09 14:45:32.090 | DEBUG    | __main__:<module>:58 - Department count samples class 0: label                   688
# department              688
# entry_date              688
# general_id              688
# normative_identifier    688
# publication_date        688
# regulatory_authority    688
# subject                 688
# subject_length          688
# subject_unique_words    688
# subject_words           688
# content                 688
# text_length             688
# text_unique_words       688
# text_words              688
# type                    688
# unique_document_id      688
# url                     688
# new_content             688
# dtype: int64
# 2023-10-09 14:49:26.219 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-09 14:49:26.221 | INFO     | __main__:training_model:214 - Training loss: 0.695085362151817
# 2023-10-09 14:49:31.232 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6912910805808173, [[0.2055144  0.05595708]
 
# 2023-10-09 14:49:31.236 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.296969696969697
# 2023-10-09 14:49:31.237 | INFO     | __main__:training_model:220 - Validation loss: 0.6912910805808173
# 2023-10-09 14:49:31.238 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.296969696969697
# 2023-10-09 14:49:56.565 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-09 14:49:56.566 | INFO     | __main__:training_model:214 - Training loss: 0.6777405473921034
# 2023-10-09 14:50:01.631 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6654380361239115, [[ 0.11723704  0.06017729]

# 2023-10-09 14:50:01.634 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6417112299465241
# 2023-10-09 14:50:01.635 | INFO     | __main__:training_model:220 - Validation loss: 0.6654380361239115
# 2023-10-09 14:50:01.636 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6417112299465241
# 2023-10-09 14:50:26.659 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-09 14:50:26.660 | INFO     | __main__:training_model:214 - Training loss: 0.6102634867032369
# 2023-10-09 14:50:31.739 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.5535002922018369, [[ 0.14926945 -0.18903586]

# 2023-10-09 14:50:31.743 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.709278350515464
# 2023-10-09 14:50:31.744 | INFO     | __main__:training_model:220 - Validation loss: 0.5535002922018369
# 2023-10-09 14:50:31.745 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.709278350515464
# 2023-10-09 14:50:57.377 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-09 14:50:57.378 | INFO     | __main__:training_model:214 - Training loss: 0.45927283609354935
# 2023-10-09 14:51:02.457 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.4628704951869117, [[ 0.05869512 -0.1269123 ]

# 2023-10-09 14:51:02.461 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.7773584905660377
# 2023-10-09 14:51:02.462 | INFO     | __main__:training_model:220 - Validation loss: 0.4628704951869117
# 2023-10-09 14:51:02.463 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.7773584905660377
# 2023-10-09 14:51:28.028 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-09 14:51:28.031 | INFO     | __main__:training_model:214 - Training loss: 0.3702679971853892
# 2023-10-09 14:51:33.132 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.47815626073214745, [[ 0.63872284 -0.8074425 ]

# 2023-10-09 14:51:33.135 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.7615230460921845
# 2023-10-09 14:51:33.136 | INFO     | __main__:training_model:220 - Validation loss: 0.47815626073214745
# 2023-10-09 14:51:33.137 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.7615230460921845
# 2023-10-09 14:51:39.664 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.47815626073214745, [[ 0.63872284 -0.8074425 ]

# 2023-10-09 14:51:39.666 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-09 14:51:39.667 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 242/276
# 2023-10-09 14:51:39.667 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.8768115942028986

# 2023-10-09 14:51:39.668 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-09 14:51:39.668 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 190/275
# 2023-10-09 14:51:39.669 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.6909090909090909

# 2023-10-09 14:51:39.677 | INFO     | __main__:results:157 - {'0': {'precision': 0.7400611620795107, 'recall': 0.8768115942028986, 'f1-score': 0.802653399668325, 'support': 276}, '1': {'precision': 0.8482142857142857, 'recall': 0.6909090909090909, 'f1-score': 0.7615230460921845, 'support': 275}, 'accuracy': 0.7840290381125227, 'macro avg': {'precision': 0.7941377238968982, 'recall': 0.7838603425559947, 'f1-score': 0.7820882228802548, 'support': 551}, 'weighted avg': {'precision': 0.7940395813164673, 'recall': 0.7840290381125227, 'f1-score': 0.782125546250106, 'support': 551}}
# 2023-10-09 14:51:39.679 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.7400611620795107, 'recall': 0.8768115942028986, 'f1-score': 0.802653399668325, 'support': 276}, '1': {'precision': 0.8482142857142857, 'recall': 0.6909090909090909, 'f1-score': 0.7615230460921845, 'support': 275}, 'accuracy': 0.7840290381125227, 'macro avg': {'precision': 0.7941377238968982, 'recall': 0.7838603425559947, 'f1-score': 0.7820882228802548, 'support': 551}, 'weighted avg': {'precision': 0.7940395813164673, 'recall': 0.7840290381125227, 'f1-score': 0.782125546250106, 'support': 551}}
# 2023-10-09 14:51:39.684 | INFO     | __main__:<module>:4 - total_relevant_documents_of_the_board: 683 - DPT_2
# 2023-10-09 14:51:39.693 | INFO     | __main__:<module>:5 - Total irrelevant documents of the department: 5234 - DPT_2
# 2023-10-09 14:51:39.695 | INFO     | __main__:<module>:8 - data_relevant_board: (683, 18) - DPT_2
# 2023-10-09 14:51:39.701 | INFO     | __main__:<module>:12 - data_irrelevant_board: (683, 18) - DPT_2
# 2023-10-09 14:51:39.704 | DEBUG    | __main__:<module>:18 - Department: DPT_2
# 2023-10-09 14:51:47.436 | DEBUG    | __main__:<module>:56 - Department count samples: label                   1366
# department              1366
# entry_date              1366
# general_id              1366
# normative_identifier    1366
# publication_date        1366
# regulatory_authority    1366
# subject                 1366
# subject_length          1366
# subject_unique_words    1366
# subject_words           1366
# content                 1366
# text_length             1366
# text_unique_words       1366
# text_words              1366
# type                    1366
# unique_document_id      1366
# url                     1366
# new_content             1366
# dtype: int64
# 2023-10-09 14:51:47.440 | DEBUG    | __main__:<module>:57 - Department count samples class 1: label                   683
# department              683
# entry_date              683
# general_id              683
# normative_identifier    683
# publication_date        683
# regulatory_authority    683
# subject                 683
# subject_length          683
# subject_unique_words    683
# subject_words           683
# content                 683
# text_length             683
# text_unique_words       683
# text_words              683
# type                    683
# unique_document_id      683
# url                     683
# new_content             683
# dtype: int64
# 2023-10-09 14:51:47.444 | DEBUG    | __main__:<module>:58 - Department count samples class 0: label                   683
# department              683
# entry_date              683
# general_id              683
# normative_identifier    683
# publication_date        683
# regulatory_authority    683
# subject                 683
# subject_length          683
# subject_unique_words    683
# subject_words           683
# content                 683
# text_length             683
# text_unique_words       683
# text_words              683
# type                    683
# unique_document_id      683
# url                     683
# new_content             683
# dtype: int64
# 2023-10-09 14:54:16.258 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-09 14:54:16.259 | INFO     | __main__:training_model:214 - Training loss: 0.6795911060439216
# 2023-10-09 14:54:21.239 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6530362599425845, [[0.04438454 0.34937584]

# 2023-10-09 14:54:21.242 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.7337883959044368
# 2023-10-09 14:54:21.243 | INFO     | __main__:training_model:220 - Validation loss: 0.6530362599425845
# 2023-10-09 14:54:21.244 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.7337883959044368
# 2023-10-09 14:54:46.513 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-09 14:54:46.514 | INFO     | __main__:training_model:214 - Training loss: 0.6074686447779337
# 2023-10-09 14:54:51.542 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.5370134876834022, [[-0.17149848  0.57681185]

# 2023-10-09 14:54:51.545 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.7738317757009345
# 2023-10-09 14:54:51.547 | INFO     | __main__:training_model:220 - Validation loss: 0.5370134876834022
# 2023-10-09 14:54:51.548 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.7738317757009345
# 2023-10-09 14:55:17.048 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-09 14:55:17.049 | INFO     | __main__:training_model:214 - Training loss: 0.4572907642081932
# 2023-10-09 14:55:22.081 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.40065085680948365, [[-0.36941475  0.79914576]

# 2023-10-09 14:55:22.084 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8317214700193424
# 2023-10-09 14:55:22.086 | INFO     | __main__:training_model:220 - Validation loss: 0.40065085680948365
# 2023-10-09 14:55:22.087 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8317214700193424
# 2023-10-09 14:55:47.541 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-09 14:55:47.542 | INFO     | __main__:training_model:214 - Training loss: 0.317306344155912
# 2023-10-09 14:55:52.580 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.32522347797122264, [[-0.42766017  0.41736615]

# 2023-10-09 14:55:52.584 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8752399232245681
# 2023-10-09 14:55:52.586 | INFO     | __main__:training_model:220 - Validation loss: 0.32522347797122264
# 2023-10-09 14:55:52.587 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8752399232245681
# 2023-10-09 14:56:18.050 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-09 14:56:18.051 | INFO     | __main__:training_model:214 - Training loss: 0.20294699707516917
# 2023-10-09 14:56:23.095 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.30904533755448127, [[-0.44677892  0.4400806 ]

# 2023-10-09 14:56:23.099 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8805970149253731
# 2023-10-09 14:56:23.100 | INFO     | __main__:training_model:220 - Validation loss: 0.30904533755448127
# 2023-10-09 14:56:23.101 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8805970149253731
# 2023-10-09 14:56:29.597 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.30904533755448127, [[-0.44677892  0.4400806 ]

# 2023-10-09 14:56:29.599 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-09 14:56:29.600 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 247/274
# 2023-10-09 14:56:29.601 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.9014598540145985

# 2023-10-09 14:56:29.602 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-09 14:56:29.603 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 236/273
# 2023-10-09 14:56:29.604 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.8644688644688645

# 2023-10-09 14:56:29.611 | INFO     | __main__:results:157 - {'0': {'precision': 0.8697183098591549, 'recall': 0.9014598540145985, 'f1-score': 0.8853046594982079, 'support': 274}, '1': {'precision': 0.8973384030418251, 'recall': 0.8644688644688645, 'f1-score': 0.8805970149253731, 'support': 273}, 'accuracy': 0.8829981718464351, 'macro avg': {'precision': 0.88352835645049, 'recall': 0.8829643592417316, 'f1-score': 0.8829508372117905, 'support': 547}, 'weighted avg': {'precision': 0.8835031095645826, 'recall': 0.8829981718464351, 'f1-score': 0.8829551403603945, 'support': 547}}
# 2023-10-09 14:56:29.613 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.8697183098591549, 'recall': 0.9014598540145985, 'f1-score': 0.8853046594982079, 'support': 274}, '1': {'precision': 0.8973384030418251, 'recall': 0.8644688644688645, 'f1-score': 0.8805970149253731, 'support': 273}, 'accuracy': 0.8829981718464351, 'macro avg': {'precision': 0.88352835645049, 'recall': 0.8829643592417316, 'f1-score': 0.8829508372117905, 'support': 547}, 'weighted avg': {'precision': 0.8835031095645826, 'recall': 0.8829981718464351, 'f1-score': 0.8829551403603945, 'support': 547}}
# 2023-10-09 14:56:29.621 | INFO     | __main__:<module>:4 - total_relevant_documents_of_the_board: 600 - DPT_9
# 2023-10-09 14:56:29.628 | INFO     | __main__:<module>:5 - Total irrelevant documents of the department: 3619 - DPT_9
# 2023-10-09 14:56:29.630 | INFO     | __main__:<module>:8 - data_relevant_board: (600, 18) - DPT_9
# 2023-10-09 14:56:29.636 | INFO     | __main__:<module>:12 - data_irrelevant_board: (600, 18) - DPT_9
# 2023-10-09 14:56:29.640 | DEBUG    | __main__:<module>:18 - Department: DPT_9
# 2023-10-09 14:56:32.032 | DEBUG    | __main__:<module>:56 - Department count samples: label                   1200
# department              1200
# entry_date              1200
# general_id              1200
# normative_identifier    1200
# publication_date        1200
# regulatory_authority    1200
# subject                 1200
# subject_length          1200
# subject_unique_words    1200
# subject_words           1200
# content                 1200
# text_length             1200
# text_unique_words       1200
# text_words              1200
# type                    1200
# unique_document_id      1200
# url                     1200
# new_content             1200
# dtype: int64
# 2023-10-09 14:56:32.037 | DEBUG    | __main__:<module>:57 - Department count samples class 1: label                   600
# department              600
# entry_date              600
# general_id              600
# normative_identifier    600
# publication_date        600
# regulatory_authority    600
# subject                 600
# subject_length          600
# subject_unique_words    600
# subject_words           600
# content                 600
# text_length             600
# text_unique_words       600
# text_words              600
# type                    600
# unique_document_id      600
# url                     600
# new_content             600
# dtype: int64
# 2023-10-09 14:56:32.040 | DEBUG    | __main__:<module>:58 - Department count samples class 0: label                   600
# department              600
# entry_date              600
# general_id              600
# normative_identifier    600
# publication_date        600
# regulatory_authority    600
# subject                 600
# subject_length          600
# subject_unique_words    600
# subject_words           600
# content                 600
# text_length             600
# text_unique_words       600
# text_words              600
# type                    600
# unique_document_id      600
# url                     600
# new_content             600
# dtype: int64
# 2023-10-09 14:57:33.678 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-09 14:57:33.679 | INFO     | __main__:training_model:214 - Training loss: 0.6889874165256819
# 2023-10-09 14:57:38.092 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6636022441089153, [[ 0.22822052  0.18984413]

# 2023-10-09 14:57:38.096 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.7767857142857143
# 2023-10-09 14:57:38.097 | INFO     | __main__:training_model:220 - Validation loss: 0.6636022441089153
# 2023-10-09 14:57:38.098 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.7767857142857143
# 2023-10-09 14:58:00.623 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-09 14:58:00.624 | INFO     | __main__:training_model:214 - Training loss: 0.6229710715512434
# 2023-10-09 14:58:05.047 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.535645604133606, [[ 9.60472375e-02  4.55538571e-01]

# 2023-10-09 14:58:05.051 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8636363636363635
# 2023-10-09 14:58:05.053 | INFO     | __main__:training_model:220 - Validation loss: 0.535645604133606
# 2023-10-09 14:58:05.054 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8636363636363635
# 2023-10-09 14:58:27.787 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-09 14:58:27.789 | INFO     | __main__:training_model:214 - Training loss: 0.45351970195770264
# 2023-10-09 14:58:32.218 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3710395786911249, [[-0.21994044  0.9033027 ]

# 2023-10-09 14:58:32.222 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8610038610038611
# 2023-10-09 14:58:32.223 | INFO     | __main__:training_model:220 - Validation loss: 0.3710395786911249
# 2023-10-09 14:58:32.224 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8610038610038611
# 2023-10-09 14:58:54.805 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-09 14:58:54.807 | INFO     | __main__:training_model:214 - Training loss: 0.3084750386575858
# 2023-10-09 14:58:59.240 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.33561364840716124, [[-3.26707810e-01  1.06373823e+00]

# 2023-10-09 14:58:59.244 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8388520971302428
# 2023-10-09 14:58:59.245 | INFO     | __main__:training_model:220 - Validation loss: 0.33561364840716124
# 2023-10-09 14:58:59.246 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8388520971302428
# 2023-10-09 14:59:21.815 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-09 14:59:21.817 | INFO     | __main__:training_model:214 - Training loss: 0.2220328658198317
# 2023-10-09 14:59:26.251 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.29290504311211407, [[-0.6839593   1.4619097 ]

# 2023-10-09 14:59:26.254 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8780487804878048
# 2023-10-09 14:59:26.256 | INFO     | __main__:training_model:220 - Validation loss: 0.29290504311211407
# 2023-10-09 14:59:26.257 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8780487804878048
# 2023-10-09 14:59:32.132 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.29290504311211407, [[-0.6839593   1.4619097 ]

# 2023-10-09 14:59:32.134 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-09 14:59:32.135 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 204/240
# 2023-10-09 14:59:32.135 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.85

# 2023-10-09 14:59:32.136 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-09 14:59:32.136 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 216/240
# 2023-10-09 14:59:32.137 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.9

# 2023-10-09 14:59:32.145 | INFO     | __main__:results:157 - {'0': {'precision': 0.8947368421052632, 'recall': 0.85, 'f1-score': 0.8717948717948718, 'support': 240}, '1': {'precision': 0.8571428571428571, 'recall': 0.9, 'f1-score': 0.8780487804878048, 'support': 240}, 'accuracy': 0.875, 'macro avg': {'precision': 0.8759398496240601, 'recall': 0.875, 'f1-score': 0.8749218261413383, 'support': 480}, 'weighted avg': {'precision': 0.8759398496240601, 'recall': 0.875, 'f1-score': 0.8749218261413383, 'support': 480}}
# 2023-10-09 14:59:32.146 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.8947368421052632, 'recall': 0.85, 'f1-score': 0.8717948717948718, 'support': 240}, '1': {'precision': 0.8571428571428571, 'recall': 0.9, 'f1-score': 0.8780487804878048, 'support': 240}, 'accuracy': 0.875, 'macro avg': {'precision': 0.8759398496240601, 'recall': 0.875, 'f1-score': 0.8749218261413383, 'support': 480}, 'weighted avg': {'precision': 0.8759398496240601, 'recall': 0.875, 'f1-score': 0.8749218261413383, 'support': 480}}
# 2023-10-09 14:59:32.151 | INFO     | __main__:<module>:4 - total_relevant_documents_of_the_board: 534 - DPT_5
# 2023-10-09 14:59:32.157 | INFO     | __main__:<module>:5 - Total irrelevant documents of the department: 2594 - DPT_5
# 2023-10-09 14:59:32.159 | INFO     | __main__:<module>:8 - data_relevant_board: (534, 18) - DPT_5
# 2023-10-09 14:59:32.164 | INFO     | __main__:<module>:12 - data_irrelevant_board: (534, 18) - DPT_5
# 2023-10-09 14:59:32.167 | DEBUG    | __main__:<module>:18 - Department: DPT_5
# 2023-10-09 14:59:35.801 | DEBUG    | __main__:<module>:56 - Department count samples: label                   1068
# department              1068
# entry_date              1068
# general_id              1068
# normative_identifier    1068
# publication_date        1068
# regulatory_authority    1068
# subject                 1068
# subject_length          1068
# subject_unique_words    1068
# subject_words           1068
# content                 1068
# text_length             1068
# text_unique_words       1068
# text_words              1068
# type                    1068
# unique_document_id      1068
# url                     1068
# new_content             1068
# dtype: int64
# 2023-10-09 14:59:35.805 | DEBUG    | __main__:<module>:57 - Department count samples class 1: label                   534
# department              534
# entry_date              534
# general_id              534
# normative_identifier    534
# publication_date        534
# regulatory_authority    534
# subject                 534
# subject_length          534
# subject_unique_words    534
# subject_words           534
# content                 534
# text_length             534
# text_unique_words       534
# text_words              534
# type                    534
# unique_document_id      534
# url                     534
# new_content             534
# dtype: int64
# 2023-10-09 14:59:35.808 | DEBUG    | __main__:<module>:58 - Department count samples class 0: label                   534
# department              534
# entry_date              534
# general_id              534
# normative_identifier    534
# publication_date        534
# regulatory_authority    534
# subject                 534
# subject_length          534
# subject_unique_words    534
# subject_words           534
# content                 534
# text_length             534
# text_unique_words       534
# text_words              534
# type                    534
# unique_document_id      534
# url                     534
# new_content             534
# dtype: int64
# 2023-10-09 15:00:56.261 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-09 15:00:56.263 | INFO     | __main__:training_model:214 - Training loss: 0.6825428065799531
# 2023-10-09 15:01:00.185 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6676163801125118, [[ 2.01843604e-01  1.71931118e-01]

# 2023-10-09 15:01:00.188 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6774193548387097
# 2023-10-09 15:01:00.190 | INFO     | __main__:training_model:220 - Validation loss: 0.6676163801125118
# 2023-10-09 15:01:00.191 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6774193548387097
# 2023-10-09 15:01:20.476 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-09 15:01:20.477 | INFO     | __main__:training_model:214 - Training loss: 0.6435632563772655
# 2023-10-09 15:01:24.414 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.5964278472321374, [[ 0.21873781  0.28505254]

# 2023-10-09 15:01:24.417 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.7922912205567451
# 2023-10-09 15:01:24.418 | INFO     | __main__:training_model:220 - Validation loss: 0.5964278472321374
# 2023-10-09 15:01:24.419 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.7922912205567451
# 2023-10-09 15:01:44.681 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-09 15:01:44.683 | INFO     | __main__:training_model:214 - Training loss: 0.5557246804237366
# 2023-10-09 15:01:48.619 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.486836776137352, [[ 0.12660202  0.45665687]

# 2023-10-09 15:01:48.622 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8252631578947368
# 2023-10-09 15:01:48.624 | INFO     | __main__:training_model:220 - Validation loss: 0.486836776137352
# 2023-10-09 15:01:48.624 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8252631578947368
# 2023-10-09 15:02:08.974 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-09 15:02:08.976 | INFO     | __main__:training_model:214 - Training loss: 0.4332268081960224
# 2023-10-09 15:02:12.927 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.37060585830892834, [[ 2.47653037e-01  3.91857058e-01]

# 2023-10-09 15:02:12.930 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8509719222462203
# 2023-10-09 15:02:12.931 | INFO     | __main__:training_model:220 - Validation loss: 0.37060585830892834
# 2023-10-09 15:02:12.932 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8509719222462203
# 2023-10-09 15:02:33.213 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-09 15:02:33.214 | INFO     | __main__:training_model:214 - Training loss: 0.33645527561505634
# 2023-10-09 15:02:37.165 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.31310831968273434, [[ 3.60436618e-01  2.05713198e-01]

# 2023-10-09 15:02:37.169 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.859090909090909
# 2023-10-09 15:02:37.170 | INFO     | __main__:training_model:220 - Validation loss: 0.31310831968273434
# 2023-10-09 15:02:37.171 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.859090909090909
# 2023-10-09 15:02:42.528 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.31310831968273434, [[ 3.60436618e-01  2.05713198e-01]

# 2023-10-09 15:02:42.531 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-09 15:02:42.531 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 177/214
# 2023-10-09 15:02:42.532 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.8271028037383178

# 2023-10-09 15:02:42.533 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-09 15:02:42.534 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 189/214
# 2023-10-09 15:02:42.535 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.883177570093458

# 2023-10-09 15:02:42.542 | INFO     | __main__:results:157 - {'0': {'precision': 0.8762376237623762, 'recall': 0.8271028037383178, 'f1-score': 0.8509615384615384, 'support': 214}, '1': {'precision': 0.8362831858407079, 'recall': 0.883177570093458, 'f1-score': 0.859090909090909, 'support': 214}, 'accuracy': 0.8551401869158879, 'macro avg': {'precision': 0.8562604048015421, 'recall': 0.8551401869158879, 'f1-score': 0.8550262237762237, 'support': 428}, 'weighted avg': {'precision': 0.856260404801542, 'recall': 0.8551401869158879, 'f1-score': 0.8550262237762237, 'support': 428}}
# 2023-10-09 15:02:42.544 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.8762376237623762, 'recall': 0.8271028037383178, 'f1-score': 0.8509615384615384, 'support': 214}, '1': {'precision': 0.8362831858407079, 'recall': 0.883177570093458, 'f1-score': 0.859090909090909, 'support': 214}, 'accuracy': 0.8551401869158879, 'macro avg': {'precision': 0.8562604048015421, 'recall': 0.8551401869158879, 'f1-score': 0.8550262237762237, 'support': 428}, 'weighted avg': {'precision': 0.856260404801542, 'recall': 0.8551401869158879, 'f1-score': 0.8550262237762237, 'support': 428}}
# 2023-10-09 15:02:42.549 | INFO     | __main__:<module>:4 - total_relevant_documents_of_the_board: 439 - DPT_16
# 2023-10-09 15:02:42.556 | INFO     | __main__:<module>:5 - Total irrelevant documents of the department: 1558 - DPT_16
# 2023-10-09 15:02:42.558 | INFO     | __main__:<module>:8 - data_relevant_board: (439, 18) - DPT_16
# 2023-10-09 15:02:42.563 | INFO     | __main__:<module>:12 - data_irrelevant_board: (439, 18) - DPT_16
# 2023-10-09 15:02:42.567 | DEBUG    | __main__:<module>:18 - Department: DPT_16
# 2023-10-09 15:02:47.968 | DEBUG    | __main__:<module>:56 - Department count samples: label                   878
# department              878
# entry_date              878
# general_id              878
# normative_identifier    878
# publication_date        878
# regulatory_authority    878
# subject                 878
# subject_length          878
# subject_unique_words    878
# subject_words           878
# content                 878
# text_length             878
# text_unique_words       878
# text_words              878
# type                    878
# unique_document_id      878
# url                     878
# new_content             878
# dtype: int64
# 2023-10-09 15:02:47.972 | DEBUG    | __main__:<module>:57 - Department count samples class 1: label                   439
# department              439
# entry_date              439
# general_id              439
# normative_identifier    439
# publication_date        439
# regulatory_authority    439
# subject                 439
# subject_length          439
# subject_unique_words    439
# subject_words           439
# content                 439
# text_length             439
# text_unique_words       439
# text_words              439
# type                    439
# unique_document_id      439
# url                     439
# new_content             439
# dtype: int64
# 2023-10-09 15:02:47.975 | DEBUG    | __main__:<module>:58 - Department count samples class 0: label                   439
# department              439
# entry_date              439
# general_id              439
# normative_identifier    439
# publication_date        439
# regulatory_authority    439
# subject                 439
# subject_length          439
# subject_unique_words    439
# subject_words           439
# content                 439
# text_length             439
# text_unique_words       439
# text_words              439
# type                    439
# unique_document_id      439
# url                     439
# new_content             439
# dtype: int64
# 2023-10-09 15:04:30.378 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-09 15:04:30.381 | INFO     | __main__:training_model:214 - Training loss: 0.6813669590389028
# 2023-10-09 15:04:33.590 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6594607383012772, [[ 0.18024369  0.19684221]

# 2023-10-09 15:04:33.594 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.4452830188679245
# 2023-10-09 15:04:33.595 | INFO     | __main__:training_model:220 - Validation loss: 0.6594607383012772
# 2023-10-09 15:04:33.596 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.4452830188679245
# 2023-10-09 15:04:50.565 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-09 15:04:50.567 | INFO     | __main__:training_model:214 - Training loss: 0.6497196274645188
# 2023-10-09 15:04:53.808 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.5891619001825651, [[ 7.53373578e-02  3.42790037e-01]

# 2023-10-09 15:04:53.811 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8028169014084507
# 2023-10-09 15:04:53.813 | INFO     | __main__:training_model:220 - Validation loss: 0.5891619001825651
# 2023-10-09 15:04:53.814 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8028169014084507
# 2023-10-09 15:05:10.862 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-09 15:05:10.864 | INFO     | __main__:training_model:214 - Training loss: 0.5663757517057306
# 2023-10-09 15:05:14.111 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.4922339866558711, [[ 1.08278364e-01  2.89585948e-01]

# 2023-10-09 15:05:14.114 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8555858310626704
# 2023-10-09 15:05:14.116 | INFO     | __main__:training_model:220 - Validation loss: 0.4922339866558711
# 2023-10-09 15:05:14.117 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8555858310626704
# 2023-10-09 15:05:31.218 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-09 15:05:31.220 | INFO     | __main__:training_model:214 - Training loss: 0.4653099319514106
# 2023-10-09 15:05:34.473 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3926364276558161, [[ 8.24055895e-02  4.27274525e-01]

# 2023-10-09 15:05:34.476 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8668555240793201
# 2023-10-09 15:05:34.478 | INFO     | __main__:training_model:220 - Validation loss: 0.3926364276558161
# 2023-10-09 15:05:34.478 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8668555240793201
# 2023-10-09 15:05:51.532 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-09 15:05:51.533 | INFO     | __main__:training_model:214 - Training loss: 0.3822500670657438
# 2023-10-09 15:05:54.786 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.30035616271197796, [[-2.78608382e-01  8.52224231e-01]

# 2023-10-09 15:05:54.789 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8948787061994609
# 2023-10-09 15:05:54.790 | INFO     | __main__:training_model:220 - Validation loss: 0.30035616271197796
# 2023-10-09 15:05:54.791 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8948787061994609
# 2023-10-09 15:05:59.379 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.30035616271197796, [[-2.78608382e-01  8.52224231e-01]

# 2023-10-09 15:05:59.381 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-09 15:05:59.382 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 147/176
# 2023-10-09 15:05:59.383 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.8352272727272727

# 2023-10-09 15:05:59.383 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-09 15:05:59.384 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 166/176
# 2023-10-09 15:05:59.385 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.9431818181818182

# 2023-10-09 15:05:59.392 | INFO     | __main__:results:157 - {'0': {'precision': 0.9363057324840764, 'recall': 0.8352272727272727, 'f1-score': 0.8828828828828829, 'support': 176}, '1': {'precision': 0.8512820512820513, 'recall': 0.9431818181818182, 'f1-score': 0.8948787061994609, 'support': 176}, 'accuracy': 0.8892045454545454, 'macro avg': {'precision': 0.8937938918830639, 'recall': 0.8892045454545454, 'f1-score': 0.8888807945411719, 'support': 352}, 'weighted avg': {'precision': 0.8937938918830639, 'recall': 0.8892045454545454, 'f1-score': 0.8888807945411717, 'support': 352}}
# 2023-10-09 15:05:59.394 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.9363057324840764, 'recall': 0.8352272727272727, 'f1-score': 0.8828828828828829, 'support': 176}, '1': {'precision': 0.8512820512820513, 'recall': 0.9431818181818182, 'f1-score': 0.8948787061994609, 'support': 176}, 'accuracy': 0.8892045454545454, 'macro avg': {'precision': 0.8937938918830639, 'recall': 0.8892045454545454, 'f1-score': 0.8888807945411719, 'support': 352}, 'weighted avg': {'precision': 0.8937938918830639, 'recall': 0.8892045454545454, 'f1-score': 0.8888807945411717, 'support': 352}}
